<a href="https://colab.research.google.com/github/LiviaCavalcanti/RecInfo/blob/master/indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import stem
import operator


In [0]:
url = "https://raw.githubusercontent.com/LiviaCavalcanti/RecInfo/master/lab02/results.csv"
original_data = pd.read_csv(url)
amount_doc = len(original_data.text)
original_data.head()

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


In [0]:
def tokenize(document, stem = False):
  tokens = [token.lower() if isinstance(token, str) else token for token in re.split(r'\W+', document)]
  if(stem):
    stemmer = stem.snowball.SnowballStemmer('portuguese')  
    return  [stemmer.stem(word) for word in tokens]
  else:
    return tokens
  

### Parte 1
_Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv. (1 pt)_

---



Para a construção dos índices, foi utilizado dicionário como estrutura de dados. À medida que os _tokens_ são gerados para cada um dos documentos, o dicionário é atulizado com o novos _tokens_ como chaves e, como valores, são construídos novos dicionários. Cada um desses dicionários é contruído para que a chave seja o _id_ do documento e o valor seja a frequência do _token_ naquele documento.

In [0]:
def build_index(documents, stem=False):
  indexes = dict()
  
  for doc_index in range(len(documents)):
    doc_tokens = tokenize(documents[doc_index], stem)

    for token in doc_tokens:
      ## o token já foi visto antes
      if not token in indexes:
        indexes[token] = {str(doc_index): 1}
      else:
        if not str(doc_index) in indexes[token]:
          indexes[token][str(doc_index)] = 1
        else:

          indexes[token][str(doc_index)] += 1


  return indexes

In [0]:
## dicionario de tokens com respectivas frequências em cada documento, também em dicionário
index_freq = build_index(original_data["text"])
doc_freq_df = pd.DataFrame(index_freq.items(), columns=["token", "doc_freq"])

In [0]:
doc_freq_df.head()

,token,doc_freq
0,a,"{'0': 27, '1': 16, '2': 38, '3': 24, '4': 26, ..."
1,juíza,"{'0': 2, '1': 1}"
2,federal,"{'0': 2, '1': 2, '2': 1, '5': 1, '6': 3, '14':..."
3,ivani,"{'0': 1, '1': 1}"
4,silva,"{'0': 3, '1': 1, '5': 1, '13': 2, '25': 1, '72..."


### Parte 2
*Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18). (2 pts)*
  
  1. *Defina 5 consultas de um termo somente.*
  2. *Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).*
  3. *Dê evidências de que sua implementação está correta.*
  4. *Compare os tempos médios de execução e uso de memória de cada algoritmo.*

---



O uso de dicionário também foi feito aqui para a geração dos resultados. Outro fator que diferenciou, ligeiramente, da implementação base do livro foi o conjunto de tokens aqui usados já vir com o _score_ de cada documento em relação ao _token_. Ficando, com isso, o cálculo do _score_ resumido a soma desses valores.



In [0]:
def term_at_a_time_retrieval(documents, query, index, n_doc_retrieve):
  query_freq = index[index["token"].isin(query)][["doc_freq"]]
  
  token_score = dict()
  for index, row in query_freq.iterrows():
    freq_dic = row.doc_freq
    for item in freq_dic:
      if item in token_score.keys():
        token_score[item] += freq_dic[item]
      else:
        token_score[item] = freq_dic[item]
        
  doc_score = sorted(sorted(token_score.items(), key=operator.itemgetter(0)), key=lambda tup: tup[1], reverse=True)
  doc_score = [i for i in doc_score if i[1] != 0]

  k_doc = []
  for elem in doc_score[:n_doc_retrieve]:
    k_doc.append(documents.loc[int(elem[0])])
    
  return k_doc

In [0]:
def doc_at_a_time_retrieval(original_data, query, index, n_doc_retrieve):
  
  query_freq = index[index["token"].isin(query)][["doc_freq"]]
  doc_indexes = original_data[["text"]].index.values
  
  dic_score = dict()
  for doc in doc_indexes:
    dic_score[str(doc)] = 0
    for index, row in query_freq.iterrows():
      if str(doc) in row["doc_freq"].keys():
        dic_score[str(doc)] += row["doc_freq"][str(doc)]
  doc_score = sorted(sorted(dic_score.items(), key=operator.itemgetter(0)), key=lambda tup: tup[1], reverse=True)
  
  doc_score = [i for i in doc_score if i[1] != 0]
  k_doc = []
  for elem in doc_score[:n_doc_retrieve]:
    k_doc.append(original_data[["text"]].loc[int(elem[0])])
  return k_doc    

Fazendo um teste simples dessas implementações para os termos, aleatoriamente escolhidos e que têm pelo menos em dez documentos: "justiça", "polícia", "política", "bolsonaro", "apesar", verificou-se que o documentos retornados são os mesmos e na mesma ordem para todos os termos.

Os _ids_ dos documentos relacionados a cada termo podem ser visto mais adiante.

In [0]:
queries_list = ["justiça", "polícia", "política", "bolsonaro", "apesar"]
results = {"query": [], "doc retrieval": [], "term retrieval": []}
for query in queries_list:
  results["query"].append(query)
  results["doc retrieval"].append(doc_at_a_time_retrieval(original_data, [query], doc_freq_df, 10))
  results["term retrieval"].append(term_at_a_time_retrieval(original_data[["text"]], [query], doc_freq_df, 10))

In [0]:
dfs = []
for query_index in range(len(queries_list)):
  print(queries_list[query_index], list(pd.DataFrame(results["doc retrieval"][query_index]).index) == list(pd.DataFrame(results["term retrieval"][query_index]).index))
  dfs.append(pd.DataFrame(data={"query": queries_list[query_index], "doc retrieval": list(pd.DataFrame(results["doc retrieval"][query_index]).index), "term retrieval": list(pd.DataFrame(results["term retrieval"][query_index]).index)}))
  

justiça True
polícia True
política True
bolsonaro True
apesar True


In [0]:
result_comparision = pd.concat(dfs)
result_comparision

,query,doc retrieval,term retrieval
0,justiça,161,161
1,justiça,65,65
2,justiça,165,165
3,justiça,28,28
4,justiça,103,103
5,justiça,203,203
6,justiça,208,208
7,justiça,216,216
8,justiça,46,46
9,justiça,72,72


Como pode ser visto a partir das verificações de uso de memória e de tempo de execução, para o formato de dados utilizado a recuperação por termo é mais eficiente. Já que, apesar de utilizarem a mesma quantidade de memória, basear-se em documento demanda mais tempo.

Isso, provavelmente, ocorre pela maior quantidade de consultas que precisam ser feitas. A recuperação baseada em documento precisa percorrer todos os documentos para atualizar o _score_, enquanto baseando-se por termo, precisa-se percorrer, apenas, os documentos que contêm o _tokens_ de interesse.

In [0]:
pip install memory_profiler

     |████████████████████████████████| 40kB 3.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/f0/ff/63/fdbff3f1e1b76ad4eae491dd5b190902906b093e93eb86dd5a
Successfully built memory-profiler


In [0]:
%load_ext memory_profiler

**Tempo de execução**

In [0]:
%%timeit
doc_at_a_time_retrieval(original_data, ["apesar"], doc_freq_df, 10)

10 loops, best of 3: 56.5 ms per loop


In [0]:
%%timeit
term_at_a_time_retrieval(original_data[["text"]], ["apesar"], doc_freq_df, 10)

100 loops, best of 3: 5.39 ms per loop


**Uso de memória**

In [0]:
%memit
_ = term_at_a_time_retrieval(original_data[["text"]], ["apesar"], doc_freq_df, 10)

peak memory: 192.51 MiB, increment: 0.00 MiB


In [0]:
%memit
_ = doc_at_a_time_retrieval(original_data, ["apesar"], doc_freq_df, 10)

peak memory: 192.51 MiB, increment: 0.00 MiB


### Parte 3

*Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21). (2 pts)*

  1. *Defina 5 consultas conjuntivas (AND).*
  2. *Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).*
  3. *Dê evidências de que sua implementação está correta.*

Como a abordagem a partir de dicionários continuou a ser utilizada por uniformidade, foi necessário usar uma função auxiliar, `dic_split`, para que a remoção de elementos do dicionários fosse realizada em conformidade do algoritmo no livro.

Comparando os resultados individuais, obtidos a partir da recuperação termo a termo, observa-se que eles são iguais aos da recuperação conjuntiva termo a termo aqui implementada. É importante lembrar que para fazer essa comparação deve-se considerar o _score_ de todos os documentos, já que não se sabe em quais documentos a interseção entre os dois termos ocorre.

In [0]:
def dic_split(dic, previous_value, actual_value):
  array_dic = list(dic.items())
  init_index = -1
  last_index = len(array_dic)
  for item_index in range(len(array_dic)):
    
    if array_dic[item_index][0] == previous_value:
      init_index = item_index
    if array_dic[item_index][0] == actual_value:
      last_index = item_index
  
  if last_index < len(array_dic)-1:
    del array_dic[init_index:last_index]
    q = dict(array_dic)
    last_index = init_index + 1
    l=array_dic[last_index][0]

  else:
    del array_dic[init_index:last_index]
    q = dict(array_dic)
    if len(array_dic)>0:
      l=array_dic[init_index-1]
    else:
      last_index = None
      l=None
  return q, l

In [0]:
def term_at_time_conjuntive_retrival(doc_freq_df, query, n_doc_retrieve):
  query_freq = doc_freq_df[doc_freq_df["token"].isin(query)]
  token_score = dict()
  fist_time_flag = True
  for index, row in query_freq.iterrows():
    freq_dic = row.doc_freq
    for item in freq_dic:

      if fist_time_flag:
        if item in token_score.keys():
          token_score[item] += freq_dic[item]
        else:
          token_score[item] = freq_dic[item]
      else:
        if item in token_score.keys():
          token_score[item] += freq_dic[item]
          token_score, previous_doc = dic_split(token_score, previous_doc, item)

    if not fist_time_flag and previous_doc not in freq_dic.keys():
      token_score, previous_doc = dic_split(token_score, previous_doc, list(token_score.items())[-1])
    if fist_time_flag:
      fist_time_flag = False
    previous_doc = list(token_score.keys())[0]
  
  doc_score = sorted(sorted(token_score.items(), key=operator.itemgetter(0)), key=lambda tup: tup[1], reverse=True)
  
  doc_score = [i for i in doc_score if i[1] != 0]
  k_doc = []
  for elem in doc_score[:n_doc_retrieve]:
    k_doc.append(original_data[["text"]].loc[int(elem[0])])
  return k_doc


In [0]:
query1 = pd.DataFrame(term_at_time_conjuntive_retrival(doc_freq_df, ["polícia", "justiça"], 10))
query1

,text
150,Quando soube que havia sido assassinada eu t...
165,O próximo domingo 31 de março marca 55 anos ...
240,Uma ação da Rota a tropa de elite da do Esta...
203,Dois dos sete presidentes que o Brasil teve de...
216,Nos últimos anos a (Funai) vem atuando com c...
72,O ex-terrorista italiano de extrema esquerda ...
6,Trajetória similar tiveram outros vários agent...
168,Em 18 de setembro de 1985 o procurador Julio ...


In [0]:
query2 = pd.DataFrame(term_at_time_conjuntive_retrival(doc_freq_df, ["polícia", "bolsonaro"], 10))
query2

,text
150,Quando soube que havia sido assassinada eu t...
165,O próximo domingo 31 de março marca 55 anos ...
206,A “rachadinha” – apropriação de salários de as...
41,O presidente chegou na madrugada deste doming...
139,A verdadeira casta –parte do 1% do Brasil– são...
203,Dois dos sete presidentes que o Brasil teve de...
229,“Putas feministas” gritava descontroladamente...
6,Trajetória similar tiveram outros vários agent...
72,O ex-terrorista italiano de extrema esquerda ...


In [0]:
query3 = pd.DataFrame(term_at_time_conjuntive_retrival(doc_freq_df, ["polícia", "política"], 10))
query3

,text
165,O próximo domingo 31 de março marca 55 anos ...
203,Dois dos sete presidentes que o Brasil teve de...
6,Trajetória similar tiveram outros vários agent...
149,Nos últimos anos a maioria dos comentaristas ...
216,Nos últimos anos a (Funai) vem atuando com c...
206,A “rachadinha” – apropriação de salários de as...
139,A verdadeira casta –parte do 1% do Brasil– são...


In [0]:
query4 = pd.DataFrame(term_at_time_conjuntive_retrival(doc_freq_df, ["apesar", "justiça"], 10))
query4

,text
65,O promotor especial não encontrou evidências ...
203,Dois dos sete presidentes que o Brasil teve de...
28,O que têm em comum os restos de fuzilados depo...
103,O ministro da Defesa do Uruguai Jorge Menénde...
222,O procurador da Justiça Militar aposentado Dur...
72,O ex-terrorista italiano de extrema esquerda ...
105,O chavismo prepara o terreno para uma eventual...
150,Quando soube que havia sido assassinada eu t...
156,Começou em frente ao parlamento sueco em 27 de...
225,O laudo de exames feitos em neto do descart...


In [0]:
query5 = pd.DataFrame(term_at_time_conjuntive_retrival(doc_freq_df, ["apesar", "bolsonaro"], 10))
query5

,text
150,Quando soube que havia sido assassinada eu t...
206,A “rachadinha” – apropriação de salários de as...
114,Assim como seis de Bolsonaro designou um mil...
234,Assim que chegar a Brasília (PSL) que declar...
204,O entrou em uma espiral de ataques internos. ...
104,O presidente do Brasil se tornou nesta segu...
217,As enormes expectativas que acompanharam a sól...
235,A aprovação do Governo de (PSL) que completa...
100,O primeiro-ministro israelense foi o convidado...
237,“Isto aqui é uma psicanálise coletiva” diz Ad...


In [0]:
# query1.isin(term_at_a_time_retrieval(original_data[["text"]], ["apesar"], doc_freq_df, 10))
import itertools

list1 = list(itertools.chain(term_at_a_time_retrieval(original_data[["text"]], ["polícia"], doc_freq_df, doc_total), term_at_a_time_retrieval(original_data[["text"]], ["justiça"], doc_freq_df, doc_total)))
list2 = list(itertools.chain(term_at_a_time_retrieval(original_data[["text"]], ["polícia"], doc_freq_df, doc_total), term_at_a_time_retrieval(original_data[["text"]], ["bolsonaro"], doc_freq_df, doc_total)))
list3 = list(itertools.chain(term_at_a_time_retrieval(original_data[["text"]], ["polícia"], doc_freq_df, doc_total), term_at_a_time_retrieval(original_data[["text"]], ["política"], doc_freq_df, doc_total)))
list4 = list(itertools.chain(term_at_a_time_retrieval(original_data[["text"]], ["justiça"], doc_freq_df, doc_total), term_at_a_time_retrieval(original_data[["text"]], ["apesar"], doc_freq_df, doc_total)))
list5 = list(itertools.chain(term_at_a_time_retrieval(original_data[["text"]], ["apesar"], doc_freq_df, doc_total), term_at_a_time_retrieval(original_data[["text"]], ["bolsonaro"], doc_freq_df, doc_total)))


In [0]:
set(query1.index).issubset(pd.DataFrame(list1).index), set(query2.index).issubset(pd.DataFrame(list2).index), set(query3.index).issubset(pd.DataFrame(list3).index), set(query4.index).issubset(pd.DataFrame(list4).index), set(query5.index).issubset(pd.DataFrame(list5).index)

(True, True, True, True, True)